In [1]:
import numpy as np
import pandas as pd
import csv

In [2]:
# to read csv data into a list
def read_csv(filename):
    # open new lists for items in .csv file
    JW_IDs = []
    ECKs = []
    B_nums = []
    Genes = []
    Sols = []       #from Niwa et.al, 2009
    Yields = []     #yield from cell free system /μM

    # go through file, line by line
    with open(filename, encoding='utf-8-sig') as f:
        for row in csv.reader(f, skipinitialspace=True):    #return values are list of values in .csv(no "comma")
            if row[0] != 'JW_ID':   #根据csv文件表头定义, 跳过表头行
                # add data to existing lists
                JW_IDs.append(row[0])
                ECKs.append(row[1])
                B_nums.append(row[2])
                Genes.append(row[3])  
                Sols.append(row[6]) 
                Yields.append(row[7])   
    f.close()
    return JW_IDs, ECKs, B_nums, Genes, Sols, Yields

In [3]:
# Inputing the filename of the .csv to process
JW_IDs, ECKs, B_nums, Genes, Sols, Yields = read_csv("all_data_esol.csv")
print(JW_IDs[0], ECKs[0], B_nums[0], Genes[0], Sols[0], Yields[0], len(Sols))
#判断是否有序列的Sol一项为空，结果发现有只有3173条数据有Sol
print(Sols[4131]== "",type(Sols[0]))

JW0002 ECK0003 b0003 thrB 32 2.3 4132
True <class 'str'>


In [4]:
# loading data in the eSol->Uniprot
JW_U = []       #Unoprot mapping后的JW_ID list
Uniprot_ID = []
PDB = []

filename = "Uniprot/eSol_Uniprot_mapping.csv"
# go through file, line by line
with open(filename, encoding='utf-8-sig') as f:
    for row in csv.reader(f, skipinitialspace=True):    #return values are list of values in .csv(no "comma")
        if row[0] != 'JW_ID':   #根据csv文件表头定义, 跳过表头行
            # add data to existing lists
            JW_U.append(row[0])
            Uniprot_ID.append(row[1])
            PDB.append(row[6])     
f.close()

In [5]:
def parse_fasta(filename):
    '''function to parse fasta'''
    # create empty lists to append names/seqs
    names =  []
    seqs = []
    # open file
    lines = open(filename,'r')
    # go through file, line by line
    for line in lines:
        # remove linebreak
        line = line.rstrip()
        # if the first character is ">"
        if line[0] == ">":
            #save names
            names.append(line[1:7])        #直接获取JW_IDs
            #start empty string
            seqs.append("")
        else:
            #add MSA to existing string (why from -1?)
            seqs[-1] += line
    #close file
    lines.close()
    return names, seqs
# return values are lists for names and seqs

def save_csv(filename, item1, item2):
    # open file
    new_csv = open(filename, 'w')
    # for each item
    for i1, i2 in zip(item1, item2):
        new_csv.write(i1 +","+ i2 +"\n")          #\n 换行
    # close
    new_csv.close()


In [6]:
# 处理fasta数据——将eSol的序列添加进来
# Inputing the filename of the fasta to process
JW_names, seqs = parse_fasta("ecoli_W3110.faa")
print(JW_names[1], seqs[1], len(JW_names))

JW0001 MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNHLVAMIEKTISGQDALPNISDAERIFAELLTGLAAAQPGFPLAQLKTFVDQEFAQIKHVLHGISLLGQCPDSINAALICRGEKMSIAIMAGVLEARGHNVTVIDPVEKLLAVGHYLESTVDIAESTRRIAASRIPADHMVLMAGFTAGNEKGELVVLGRNGSDYSAAVLAACLRADCCEIWTDVDGVYTCDPRQVPDARLLKSMSYQEAMELSYFGAKVLHPRTITPIAQFQIPCLIKNTGNPQAPGTLIGASRDEDELPVKGISNLNNMAMFSVSGPGMKGMVGMAARVFAAMSRARISVVLITQSSSEYSISFCVPQSDCVRAERAMQEEFYLELKEGLLEPLAVTERLAIISVVGDGMRTLRGISAKFFAALARANINIVAIAQGSSERSISVVVNNDDATTGVRVTHQMLFNTDQVIEVFVIGVGGVGGALLEQLKRQQSWLKNKHIDLRVCGVANSKALLTNVHGLNLENWQEELAQAKEPFNLGRLIRLVKEYHLLNPVIVDCTSSQAVADQYADFLREGFHVVTPNKKANTSSMDYYHQLRYAAEKSRRKFLYDTNVGAGLPVIENLQNLLNAGDELMKFSGILSGSLSYIFGKLDEGMSFSEATTLAREMGYTEPDPRDDLSGMDVARKLLILARETGRELELADIEIEPVLPAEFNAEGDVAAFMANLSQLDDLFAARVAKARDEGKVLRYVGNIDEDGVCRVKIAEVDGNDPLFKVKNGENALAFYSHYYQPLPLVLRGYGAGNDVTAAGVFADLLRTLSWKLGV 4316


In [7]:
#建立空list存放按照JW_ID排序后的seqs
seq_withJW = [None]*len(JW_IDs)
#将ecoli_W3110.faa文件中的seqs按照JW_ID的顺序对应起来，该文件包含的数据多于eSol
for JID, seq in zip(JW_names, seqs):
    #根据JW_ID将seqs填入新的表中,未被搜索到的值为None
    try: 
        i = JW_IDs.index(JID)
        if seq_withJW[i] == None:      #一个JW_ID只写入一次
            seq_withJW[i] = seq
        else:
            print(i, JID, seq)  
    except ValueError:
        print("Warning: " + JID + " is not in the eSol database")

In [8]:
#结果显示fasta中的seq能和完全覆盖所有eSol有数据的JW_ID
count = 0
for id in seq_withJW:
    if  id != None:
        count += 1
print(count, len(seq_withJW))

4132 4132


In [9]:
#建立空list存放按照JW_IDs排序后的Uniprot mapping
UID_withJW = [None]*len(JW_IDs)
PDB_withJW = [None]*len(JW_IDs)

#将Uniprot mapping得到的数据和按照原始JW_IDs的顺序对应起来
for ID, UID, P in zip(JW_U, Uniprot_ID, PDB):
    #根据JW_IDs中的index将Uniprot_ID, PDB, seq填入新的表中,未被mapping到的值为None
    i = JW_IDs.index(ID)
    if UID_withJW[i] == None:      #一个JW_ID只写入一次，JW4065仅保留第一个P0DP69(phne1,22.8kDa)
        UID_withJW[i] = UID
        PDB_withJW[i] = P
    else:
        print(i, ID, UID, P)        

# 判断原始JW_ID数量、成功被mapping的ID数量、和加入新表的数量是否一致
print(len(JW_IDs), len(JW_U), sum(n1 is not None for n1 in UID_withJW), sum(n2 is not None for n2 in PDB_withJW))

# use masks to find the not mapped JW_IDs, 提取没有被mapping到的JW_IDs, 
UID_withJW_array = np.array(list(UID_withJW))       #转化为了能够提取mask
JW_IDs_array = np.array(JW_IDs)
Genes_array = np.array(Genes)
No_mapping = (UID_withJW_array == None)
JWID_nomp = JW_IDs_array[No_mapping]
gene_nomp = Genes_array[No_mapping]
#print(len(JWID_nomp), JWID_nomp)

#save no_mapping results into .csv
filename = "No_mapping.csv"
save_csv(filename, JWID_nomp, gene_nomp)

2901 JW4065 P0DP70 
4132 4049 4048 4048


In [10]:
# updating the no mapping data into UID_withJW (RE_map.csv)

# loading data in the RE_map.csv
JW_Re = []       #Unoprot mapping后的JW_ID list
Uniprot_Re = []
PDB_Re = []

filename = "Uniprot/RE_map.csv"
# go through file, line by line
with open(filename, encoding='utf-8-sig') as f:
    for row in csv.reader(f, skipinitialspace=True):    #return values are list of values in .csv(no "comma")
        if row[0] != 'JW_ID':   #根据csv文件表头定义, 跳过表头行
            # add data to existing lists
            JW_Re.append(row[0])
            Uniprot_Re.append(row[1])
            PDB_Re.append(row[6])     
f.close()
print(len(JW_Re))

# 再次将Re_map中的Uniprot ID等数据添加到原始数据表中
count = 0
for ID, UID, P in zip(JW_Re, Uniprot_Re, PDB_Re):
    #根据JW_IDs中的index将Uniprot_ID, PDB, seq填入新的表中,未被mapping到的值为None
    i = JW_IDs.index(ID)
    if UID_withJW[i] == None:      #一个JW_ID只写入一次
        UID_withJW[i] = UID
        PDB_withJW[i] = P
        count += 1
    else:
        print(i, ID, UID, P)   

54


根据输出，发现Uniprot mapping结果中有一个序列重复（同一个JW_ID对应了2个uniprot_ID）
上述分析发现，重复为JW4065，对应Uniprot编号分别为：P0DP69/70，无PDB编号，基因名分别为phne1/2，根据eSol中标记的Mw可以推断为第一个
已经在UID_withJW和PDB_withJW中更新
此时得到的新eSol_datasheet为:
[JW_IDs, UID_withJW, Sols, ECKs, B_nums, Genes, Yields, PDB_withJW, seq_withJW]

此外，仅有4049条序列能在Uniprot对应上，需要进一步完善

In [11]:
# construct dataframe df_eSol, which contains items with Sol values

JW_array = np.array(JW_IDs)
UID_array = np.array(UID_withJW)
Sol_array = np.array(Sols)
Gene_array = np.array(Genes)
seq_array = np.array(seq_withJW)
PDB_array = np.array(PDB_withJW)
#print(len(JW_array), len(UID_array), len(Sol_array), len(Gene_array), len(seq_array), len(PDB_array))

# construct dataframe
d = {'JW_IDs': JW_array, 'Uniprot_ID': UID_array, 'Sol':Sol_array, "Genes": Gene_array, "Sequence": seq_array, "PDB": PDB_array}
df_eSol = pd.DataFrame(data = d)
df_eSol_sol = df_eSol[df_eSol.Sol != ""]
print(len(df_eSol.JW_IDs), len(df_eSol_sol.JW_IDs))

df_training = df_eSol_sol[df_eSol_sol['Uniprot_ID'].notnull()]
print(len(df_training.JW_IDs))


4132 3173
3151


In [12]:
#output into .csv file
df_training["Sol"] = df_training.Sol.astype(float)/100
df_eSol_output = df_training.reset_index(drop=True)
df_eSol_output[["Genes", "Uniprot_ID", "Sol"]].to_csv("eSol_output_geneNames.csv", index = False, encoding= 'utf-8')


d:\Anaconda\envs\Research_py37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
print(len(df_eSol_output.JW_IDs))
print(df_eSol_output.loc[1381:1392])
df_eSol_output.tail()


3151
      JW_IDs Uniprot_ID   Sol Genes  \
1381  JW1163     P0A734  1.02  minE   
1382  JW1166     P76001  0.78  ycgJ   
1383  JW1167     P76002  0.73  ycgK   
1384  JW1168     P0AB43  1.08  ycgL   
1385  JW5180     P0A8L5  0.86  ycgN   
1386  JW1172     P0AG11  0.66  umuD   
1387  JW1184     P76011  0.16  ymgE   
1388  JW1185     P76012  0.88  ycgY   
1389  JW1196     P0AB49  0.73  ychH   
1390  JW1204     Q46755  0.25  ychQ   
1391  JW1208     P0AE63  1.12  chaB   
1392  JW1210     P0AB52  0.92  ychN   

                                               Sequence                   PDB  
1381  MALLDFFLSRKKNTANIAKERLQIIVAERRRSDAEPHYLPQLRKDI...       1EV0;3R9I;3R9J;  
1382  MNMMRIFYIGLSGVGMMFSSMASGNDAGGLQSPACGVVCDPYICVN...                        
1383  MKIKSIRKAVLLLALLTSTSFAAGKNVNVEFRKGHSSAQYSGEIKG...       4DXZ;4DY3;4G9S;  
1384  MPKPGILKSKSMFCVIYRSSKRDQTYLYVEKKDDFSRVPEELMKGF...                        
1385  MSDVPFWQSKTLDEMSDAEWESLCDGCGQCCLHKLMDEDTDEIYFT...                        
1386  M

,JW_IDs,Uniprot_ID,Sol,Genes,Sequence,PDB
3146,JW5749,P0ADE2,0.78,ytfK,MKIFQRYNPLQVAKYVKILFRGRLYIKDVGAFEFDKGKILIPKVKD...,
3147,JW4181,P0AE48,0.98,ytfP,MRIFVYGSLRHKQGNSHWMTNAQLLGDFSIDNYQLYSLGHYPGAVP...,1XHS;
3148,JW4184,P33647,0.94,chpB,MVKKSEFERGDIVLVGFDPASGHEQQGAGRPALVLSVQAFNQLGMT...,
3149,JW4196,P0A9N8,0.69,nrdG,MNYHQYYPVDIVNGPGTRCTLFVSGCVHECPGCYNKSTWRVNSGQP...,
3150,JW5755,P0AF93,0.76,yjgF,MSKTIATENAPAAIGPYVQGVDLGNMIITSGQIPVNPKTGEVPADV...,1QU9;
